**Question 3**

Use MNISTdataset and follow below instructions to solve this question.
• Create a AutoEncoder with following instructions:
– Create feed forward Neural Network as follows:
∗ Input layer : (input = 784, output = 512,activation = ReLU)
∗ Hidden layer : (input = 512, output = 128, activation = ReLU)
∗ Latent Space : (input = 128,output = 64, activation = ReLU)
∗ Hidden layer : (input = 64, output = 128,activation = ReLU)
∗ Hidden layer :(input = 128,output = 512,activation = ReLU)
∗ Output layer : (input = 512, output = ?,activation = ReLU)

– Use appropriate loss function
– Use Adam optimizer to optimize the loss function with proper learn-
ing rate.
– Use training data to train the autoencoder
– After Training autoEncoder remove the decoder from the autoEn-
coder architecture.

• Create Classifcation Model called MNIST Classifcation Model with
following configuration.
• Use the encoder and then argument it with the following.
– Input Layer :(input = 64 , output = 32 , activation = ReLU)
– output Layer : (input = 32 , output = ? , activation = Softmax)
• Use MultiClass Cross Entropy loss function and Adam optimizer to
optimize the loss function with appropriate learning rate.

• Train MNIST Classifcation Model using training dataset.Plot epoch-
wise training loss.
• Test using testing dataset and report accuracy and classwise accuracy.

In [1]:
import numpy as np
import gzip
import sys
import pickle as cPickle
from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from tensorflow.keras.layers import BatchNormalization
from array import array

#Q3
#unpacking the images
def images_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        image_count = int.from_bytes(f.read(4), 'big')
        row_count = int.from_bytes(f.read(4), 'big')
        column_count = int.from_bytes(f.read(4), 'big')

        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8)\
            .reshape((image_count, row_count, column_count))
        return images

def labels_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        label_count = int.from_bytes(f.read(4), 'big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

train_images = images_file_read("/content/drive/MyDrive/train-images-idx3-ubyte.gz")
train_labels = labels_file_read("/content/drive/MyDrive/train-labels-idx1-ubyte.gz")
test_imagesall = images_file_read("/content/drive/MyDrive/t10k-images-idx3-ubyte.gz")
test_labelsall = labels_file_read("/content/drive/MyDrive/t10k-labels-idx1-ubyte.gz")

# dividing into validation set and test set
val_images = test_imagesall[0:500,:,:]
val_labels = test_labelsall[0:500]
test_images = test_imagesall[500:,:,:]
test_labels = test_labelsall[500:]

#normalizing images
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5

#flatening the images
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))
val_images = val_images.reshape((-1, 784))

In [2]:
input_img = keras.Input(shape=(784,))
input = layers.Dense(512, input_dim=784, activation='relu', trainable=True)(input_img)
encoded = layers.Dense(128, activation='relu',trainable=True)(input)
latent = layers.Dense(64, activation='relu',trainable=True)(encoded)

decoded1 = layers.Dense(128, activation='relu',trainable=True)(latent)
decoded2 = layers.Dense(512, activation='relu',trainable=True)(decoded1)
output = layers.Dense(784, activation='relu',trainable=True)(decoded2)


In [3]:
autoencoder = keras.Model(input_img, output)
opt = keras.optimizers.Adam(learning_rate=0.01)
autoencoder.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
history=autoencoder.fit(train_images, train_images,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(val_images,val_images))

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [11]:
input_img = keras.Input(shape=(784,))
input = layers.Dense(512, input_dim=784, activation='relu', trainable=False)(input_img)
encoded = layers.Dense(128, activation='relu',trainable=False)(input)
latent = layers.Dense(64, activation='relu',trainable=False)(encoded)

inp= layers.Dense(32,input_dim=64,activation='relu',trainable=True)(latent)
out= layers.Dense(10,input_dim=32,activation='softmax',trainable=True)(inp)

MNIST_Classification_Model= keras.Model(input_img,out)
opt = keras.optimizers.Adam(learning_rate=0.01)
MNIST_Classification_Model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

MNIST_Classification_Model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dense_11 (Dense)            (None, 128)               65664     
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 10)                330       
                                                                 
Total params: 478,250
Trainable params: 2,410
Non-trainable

In [12]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# Training the model.
history=MNIST_Classification_Model.fit(
  train_images,
  to_categorical(train_labels),
  validation_data=(val_images, to_categorical(val_labels)),  
  epochs=30,
  batch_size=512,
  shuffle = True,
  callbacks=[es,mc]
)


Epoch 1/30
114/118 [===========================>..] - ETA: 0s - loss: 1.4454 - accuracy: 0.5496
Epoch 1: val_accuracy improved from -inf to 0.67400, saving model to best_model.h5
118/118 [==============================] - 2s 15ms/step - loss: 1.4339 - accuracy: 0.5529 - val_loss: 1.0157 - val_accuracy: 0.6740
Epoch 2/30
114/118 [===========================>..] - ETA: 0s - loss: 0.9568 - accuracy: 0.6906
Epoch 2: val_accuracy improved from 0.67400 to 0.70000, saving model to best_model.h5
118/118 [==============================] - 2s 14ms/step - loss: 0.9562 - accuracy: 0.6907 - val_loss: 0.9169 - val_accuracy: 0.7000
Epoch 3/30
116/118 [============================>.] - ETA: 0s - loss: 0.8996 - accuracy: 0.7071
Epoch 3: val_accuracy did not improve from 0.70000
118/118 [==============================] - 2s 13ms/step - loss: 0.8997 - accuracy: 0.7071 - val_loss: 0.8980 - val_accuracy: 0.6980
Epoch 4/30
116/118 [============================>.] - ETA: 0s - loss: 0.8738 - accuracy: 0.7156


In [13]:
# Evaluating the model.
saved_model = load_model('best_model.h5')
scores = saved_model.evaluate(
  test_images,
  to_categorical(test_labels)
)

from sklearn.metrics import confusion_matrix
pred=MNIST_Classification_Model.predict(test_images) 
y_pred=np.argmax(pred,axis=1)

matrix = confusion_matrix(test_labels, y_pred)
cwa=matrix.diagonal()/matrix.sum(axis=1)

print("Classwise Accuracy:")
for i in range(0,10):
  print("Class ",i," :",cwa[i])

print('Overall accuracy:', scores[1])

297/297 [==============================] - 1s 3ms/step - loss: 0.6183 - accuracy: 0.8057
Classwise Accuracy:
Class  0  : 0.8987206823027718
Class  1  : 0.952247191011236
Class  2  : 0.7584442169907881
Class  3  : 0.7585492227979275
Class  4  : 0.8435814455231931
Class  5  : 0.665083135391924
Class  6  : 0.8655737704918033
Class  7  : 0.8304392236976507
Class  8  : 0.7419700214132763
Class  9  : 0.7130890052356021
Overall accuracy: 0.8056842088699341
